<a href="https://colab.research.google.com/github/carlaost/fundingdominance/blob/main/EmoBERT/EmoBERT_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###setup

In [1]:
!pip install transformers &> /dev/null
!pip install datasets &> /dev/null
!pip install huggingface_hub &> /dev/null
!pip install tensorflow &> /dev/null
!pip install tensorflow_hub &> /dev/null
!pip install tensorflow_text &> /dev/null
!pip install keras-tuner -q &> /dev/null

In [2]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
import datetime, os
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [13]:
import keras

In [18]:
import json

### Loading & Processing the Training Data

In [ ]:
test = pd.read_csv('https://raw.githubusercontent.com/carlaost/fundingdominance/main/EmoBERT_training/emobank_w_test.csv')
train_full = pd.read_csv('https://raw.githubusercontent.com/carlaost/fundingdominance/main/EmoBERT_training/emobank_w_train_full.csv')
train = pd.read_csv('https://raw.githubusercontent.com/carlaost/fundingdominance/main/EmoBERT_training/emobank_w_train.csv')
val = pd.read_csv('https://raw.githubusercontent.com/carlaost/fundingdominance/main/EmoBERT_training/emobank_w_val.csv')

In [ ]:
def x_y(data):

  x = data['text']
  v = data['V']
  a = data['A']
  d = data['D']
  y = {'V': v,
       'A': a,
       'D': d}
  return x, y

In [ ]:
train_x, train_y = x_y(train)
val_x, val_y = x_y(val)
test_x, test_y = x_y(test)

###model training

In [3]:
def create_model():
  preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3')
  bert = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4', trainable=False)
  
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  bert_inputs = preprocessor(text_input)
  bert_outputs = bert(bert_inputs)
  cls_embedding = bert_outputs['pooled_output']

  inter1 = tf.keras.layers.Dense(64, name='inter1')(cls_embedding)
  inter2 = tf.keras.layers.Dense(64, name='inter2')(inter1)

  output1 = tf.keras.layers.Dense(1, name='V')(inter2)
  output2 = tf.keras.layers.Dense(1, name='A')(inter2)
  output3 = tf.keras.layers.Dense(1, name='D')(inter2)

  model = tf.keras.Model(
    inputs=text_input, 
    outputs=[output1, output2, output3])
  
  optimizer='rmsprop'
  loss={'V':'mse', 'A':'mse', 'D':'mse'}
  metrics={'V':'mae', 'A':'mae', 'D':'mae'}

  model.compile(optimizer=optimizer,loss=loss,metrics=metrics)
  
  return model

In [ ]:
model = create_model()
#model.summary()

In [ ]:
def train_model(filepath):
  model = create_model()
  
  logdir = os.path.join(filepath, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  checkpoint = os.path.join(filepath, 'checkpoint')
  
  callbacks = [tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1),
               tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10),
               tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint,
                                                  monitor='val_loss',
                                                  save_best_only=True,
                                                  mode=min)]

  model.fit(x=train_x, y=train_y,
          batch_size=256,
          epochs=100,
          validation_data = (val_x, val_y), 
          callbacks=callbacks)

In [ ]:
filepath = xxx # specify where to save model training logs

In [ ]:
train_model(filepath)

In [ ]:
model.save(xxx) # save model architecture and weights

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/funding-dominance/emotion-analysis/model/emobert-large-64

###testing model

In [ ]:
model.evaluate(x=test_x, y=test_y)